# GeoDataFrame aanmaken vanuit een DataFrame met coördinaten

In deze demo zullen we het aanmaken van een ``GeoDataFrame`` (GeoPandas) illustreren op basis van een *reguliere* ``DataFrame``. In deze ``DataFrame`` zijn coördinaten verwerkt in afzonderlijke kolommen (lengteligging als **LON** en breedteligging als **LAT**). In het tweede deel worden dezelfde coördinaten voorgesteld in het WKT-formaat ([well-known text](https://en.wikipedia.org/wiki/Well-known_text)).

In beide gevallen zullen we de Vlaamse provinciehoofdsteden projecteren op een grafische voorstelling van Belgische provincies (geselecteerd op Vlaamse provincies).

In [ ]:
# Bibliotheken importeren
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Instelling om figuren wat groter af te beelden
plt.rcParams['figure.figsize'] = [15, 5]

## Geometriën uit kolommen

Laten we eerst eens kijken naar de ``DataFrame`` met Vlaamse provinciehoofdsteden met de vermelding van de naam van de stad (**STAD**), de Nederlandse (**PROV_NL**) en Engelse (**PROV_EN**) benaming van de provincie, de breedteligging (**LAT**) en de lengteligging (**LON**).

In [ ]:
df = pd.DataFrame(
    {'STAD': ['Brugge', 'Gent', 'Antwerpen', 'Leuven', 'Hasselt'],
     'PROV_NL': ['West-Vlaanderen', 'Oost-Vlaanderen', 'Antwerpen', 'Vlaams-Brabant', 'Limburg'],
     'PROV_EN': ['West Flanders', 'East Flanders', 'Antwerp', 'Flemish Brabant', 'Limburg'],
     'LAT': [51.26, 51.07, 51.21, 50.88, 50.93],
     'LON': [3.21, 3.72, 4.38, 4.70, 5.33]})

Een ``GeoDataFrame`` vereist een ``shapely``-object. We gebruiken de ``points_from_xy()``-methode uit GeoPandas om **LAT** en **LON** te converteren naar een lijst met ``shapely.Point``-objecten. Ieder object wordt vervolgens omgezet naar een ``geometry`` wanneer de ``GeoDataFrame`` wordt aangemaakt.

> De methode ``points_from_xy()`` is een 'wrapper' of vereenvoudigde methode voor de volgende code: ``[Point(x, y) for x, y in zip(df.Longitude, df.Latitude)]``)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LON, df.LAT))

Het object ``gdf`` ziet er nu als volgt uit:

In [ ]:
print(gdf.head())

Tot slot projecteren we deze Vlaamse procinciehoofdsteden op een kaartje met toevoeging van de provincies zelf.

In [ ]:
# We halen de data op uit een online repository op GitHub
url = "https://github.com/deldersveld/topojson/raw/master/countries/belgium/belgium-provinces.json"
belgium = gpd.read_file(url)

# We visualiseren enkel de provincies die voorkomen in onze lijst
ax = belgium[belgium.NAME_1.isin(gdf.PROV_EN)].plot(
    color='white', edgecolor='black')

# Tot slot plotten we de ``GeoDataFrame``.
gdf.plot(ax=ax, color='red')

plt.show()

## GeoDataFrame vanuit een dataset in het WKT formaat

We hernemen de dataset met Vlaamse provinciehoofdsteden als ``DataFrame``, waarbij de coördinaten worden gegeven in het WKT formaat.



In [ ]:
df = pd.DataFrame(
    {'STAD': ['Brugge', 'Gent', 'Antwerpen', 'Leuven', 'Hasselt'],
     'PROV_NL': ['West-Vlaanderen', 'Oost-Vlaanderen', 'Antwerpen', 'Vlaams-Brabant', 'Limburg'],
     'PROV_EN': ['West Flanders', 'East Flanders', 'Antwerp', 'Flemish Brabant', 'Limburg'],
     'COORD': ['POINT (3.21000 51.26000)', 'POINT (3.72000 51.07000)', 'POINT (4.38000 51.21000)',
               'POINT (4.70000 50.88000)', 'POINT (5.33000 50.93000)']})

Merk op dat deze weergave van geometriën dezelfde is als bij de voorstelling van de eerder aangemaakte ``GeoDataFrame`` (zie hierboven). We gebruiken de submodule ``shapely.wkt`` om deze data te parsen, dus voor de conversie naar een formaat dat compatibel is met GeoPandas. De methode voor de conversie van WKT naar daarwerkelijke geometieën is verschillend afhankelijk van de versie van GeoPandas.

In [ ]:
# Importeer de WKT submodule
from shapely import wkt

# Bepaal de te gebruiken methode afhankelijk van de versie
if (gpd.__version__ < '0.9'):
    df['geometry'] = df.COORD.apply(wkt.loads)
else:
    df['geometry'] = gpd.GeoSeries.from_wkt(df['COORD'])

De resulterende ``GeoDataFrame`` wordt in beide gevallen als volgt aangemaakt:



In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.drop('COORD', axis=1, inplace=True)
gdf.head()

Deze ``GeoDataFrame`` kunnen we opnieuw visualiseren, zoals hierboven reeds vermeld.

In [ ]:
url = "https://github.com/deldersveld/topojson/raw/master/countries/belgium/belgium-provinces.json"
belgium = gpd.read_file(url)
ax = belgium[belgium.NAME_1.isin(gdf.PROV_EN)].plot(
    color='white', edgecolor='black')
gdf.plot(ax=ax, color='red')
plt.show()